In [230]:
import praw
import csv
import re
import requests
from datetime import datetime
from datetime import timezone
import traceback
import json
import sys
import random

reddit = praw.Reddit(
    client_id="D41GnodGa6utMqgowNAd1g",
    client_secret="729_Nh1fOfQgXVUk_ADM4Oat-K9wOA",
    user_agent="NetworkAnalysis/1.0 by mommet33",
)

In [243]:
def popular_subreddit_generator():
    with open("popular_subreddits.csv", 'w', newline='') as file:
        for subreddit in reddit.subreddits.popular(limit=None):
            writer = csv.writer(file)
            writer.writerow([subreddit.display_name.lower()])
            yield subreddit.display_name.lower()

In [232]:
def extract_linked_subreddits_from_text(text, link_position):
    linked_subreddits = []
    for match in re.finditer(r"(?<!reddit\.com\/)r\/[a-zA-Z_0-9]+", text):
        linked_subreddits.append({
            "subreddit": match.group().removeprefix("r/").lower(),
            "link_positions": set([link_position]),
            "link_types": set(["text"])
        })
    return linked_subreddits

def extract_linked_subreddits_from_url(url, link_position=None):
    linked_subreddits = []
    for match in re.finditer(r"(?<=reddit\.com\/r\/)[a-zA-Z_0-9]+", url):
        linked_subreddits.append({
            "subreddit": match.group().lower(),
            "link_positions": set([link_position]),
            "link_types": set(["hyperlink"])
        })
    return linked_subreddits

def extract_linked_subreddits(string, link_position=None):
    linked_subreddits = extract_linked_subreddits_from_url(string, link_position)
    linked_subreddits += extract_linked_subreddits_from_text(string, link_position)
    
    deduplicated_links = {}
    
    subreddit_set = set()
    for link in linked_subreddits:
        subreddit = link["subreddit"]
        if subreddit not in deduplicated_links:
            deduplicated_links[subreddit] = link
            continue
        aggregated_link = deduplicated_links[subreddit]
        aggregated_link["link_positions"].union(link["link_positions"])
        aggregated_link["link_types"].union(link["link_types"])
            
    return list(deduplicated_links.values())

def get_all_linked_subreddits(submission):
    linked_subreddits = extract_linked_subreddits(submission.title, link_position="title")
    if submission.is_self:
        linked_subreddits += extract_linked_subreddits(submission.selftext, link_position="body")
    else:
        linked_subreddits += extract_linked_subreddits_from_url(submission.url, link_position="body")
    return linked_subreddits

In [233]:
submission = reddit.submission(id="17xtfr5")
title = submission.title
print(submission.subreddit.display_name)

OpenAI


In [283]:
for submission in reddit.subreddit("home").new(limit=None):
    print(submission.id)

17yq2xe
17yq2nz
17yo09b
17ynk0z
17ymvz8
17ykxi2
17yk06n
17yjwui
17yjg66
17yi9jk
17yhxky
17yhqf5
17yd142
17yc1nk
17ybqdl
17ybdoj
17yb1i5
17yawzi
17yamtw
17y9nj1
17y9mrb
17y97x6
17y7vff
17y6zki
17y6sho
17y3731
17xz07h
17xwjlq
17xuia1
17xtnq4
17xqbpf
17xq7qr
17xowzd
17xnplp
17xnkfs
17xkbxq
17xjuw5
17xii68
17xh70y
17xh6qb
17xgo64
17xfu0a
17xdtk2
17xafkd
17x9rvg
17x9a2c
17x5vht
17x4lw7
17x31yx
17wy75g
17wxuvg
17ww0ku
17wvywq
17wvnwr
17wui2h
17wsvnn
17wsq7o
17wonot
17wodw9
17wn055
17wmu81
17wl38q
17wkgb4
17wfr4d
17wdg00
17wd7ql
17wc1im
17w93v0
17w1ido
17w1ebt
17vzxql
17vzghv
17vyv93
17vyszg
17vydwi
17vy6ln
17vy1jm
17vxij9
17vw0i8
17vt0dh
17vnc1k
17vn1lj
17vmo41
17vmfki
17vmf9l
17vl4pu
17vkl9h
17vjxpz
17vjs9y
17vi5g6
17vi4cd
17vhyjs
17vhuft
17vgnub
17vfg6o
17vfcgc
17ve81a
17vdm1z
17vcx3z
17vcdsh
17vafwf
17va1u8
17v9uj0
17v93ow
17v8846
17v7s67
17v5z5k
17v5g3c
17v4l4c
17v47p6
17v47kf
17v3sb2
17v05f2
17uuf3p
17ut4fi
17usjrj
17urvaf
17urose
17urk7g
17upjxx
17uof15
17unhe7
17umq14
17uloxf
17ukp8c


In [235]:
get_all_linked_subreddits(submission)

[]

In [236]:
edges_header = ["source", "dest", "post_id", "post_date", "post_time", "upvotes", "link_in_title", "link_in_body", "is_hyperlink", "is_text_link"]

In [280]:
def write_links_to_file(submission, linked_subreddits, file):
    writer = csv.writer(file)
    for link in linked_subreddits:
        source = submission.subreddit.display_name.lower()
        dest = link["subreddit"]
        post_id = submission.id
        upvotes = submission.score
        link_in_title = "title" in link["link_positions"]
        link_in_body = "body" in link["link_positions"]
        is_hyperlink = "hyperlink" in link["link_types"]
        is_text_link = "text" in link["link_types"]
        
        created_at = utc_timestamp_to_datetime(submission.created_utc)
        
        post_date = created_at.strftime("%Y-%m-%d")
        post_time = created_at.strftime("%H:%M:%S")
        writer.writerow([source, dest, post_id, post_date, post_time, upvotes, link_in_title, link_in_body, is_hyperlink, is_text_link])

def find_links_in_submissions(subreddit, cutoff_date, save_to=None):
    all_subreddit_links = []
    
    subreddit_name = subreddit.display_name.lower()
    if save_to is not None:
        file = open(save_to, 'a', newline='')
    for submission in subreddit.new(limit=None):
        if submission.over_18: continue
        if submission.created_utc < cutoff_date:
#             created_str = utc_timestamp_to_datetime(submission.created_utc).strftime("%Y-%m-%d")
#             cutoff_str = utc_timestamp_to_datetime(cutoff_date).strftime("%Y-%m-%d")
#             print(f"over_18: {submission.over_18}, created: {created_str}, cutoff: {cutoff_str}, less: {submission.created_utc < cutoff_date}, created_utc: {submission.created_utc}, cutoff_utc: {cutoff_date}")
            break
        
        linked_subreddits = get_all_linked_subreddits(submission)
        all_subreddit_links += linked_subreddits
        
        write_links_to_file(submission, linked_subreddits, file)
    
    if file is not None:
        file.close()
    
    return all_subreddit_links

def datetime_to_utc_timestamp(dt):
    return dt.replace(tzinfo=timezone.utc).timestamp()

def utc_timestamp_to_datetime(utc):
    return datetime.utcfromtimestamp(utc)
    
class ListDict:
    def __init__(self):
        self.item_to_position = {}
        self.items = []

    def add(self, item):
        if item in self.item_to_position:
            return
        self.items.append(item)
        self.item_to_position[item] = len(self.items)-1

    def remove(self, item):
        if item not in self.item_to_position: return
        position = self.item_to_position.pop(item)
        last_item = self.items.pop()
        if position != len(self.items):
            self.items[position] = last_item
            self.item_to_position[last_item] = position
    def __len__(self):
        return len(self.items)

    def choose_random_item(self):
        return random.choice(self.items)
    
    def random_pop(self):
        item = self.choose_random_item()
        self.remove(item)
        return item

In [284]:
edges_csv = "edges.csv"
with open(edges_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(edges_header)

In [285]:
popular_subreddits = popular_subreddit_generator()
subreddit_visit_queue = ListDict()
visited_subreddits = set()

cutoff_date = datetime_to_utc_timestamp(datetime(2020, 11, 19))
while True:
    subreddit_name = None
    
    chance_to_explore_popular = 0.2
    explore_popular = random.random() < chance_to_explore_popular
    explore_popular = explore_popular or len(subreddit_visit_queue)==0
    if not explore_popular:
        subreddit_name = subreddit_visit_queue.random_pop()
    else:
        popular_subreddit = next(popular_subreddits, None)
        if popular_subreddit is not None:
            subreddit_name = popular_subreddit
        elif len(subreddit_visit_queue)>0:
            subreddit_name = subreddit_visit_queue.random_pop()
        else:
            print("Finished processing everything!")
            break
    
    subreddit_name = subreddit_name.lower()
    if subreddit_name in visited_subreddits:
        continue
    visited_subreddits.add(subreddit_name)
    
    print(f"Processing subreddit '{subreddit_name}'{' (from popular)' if explore_popular else ''}")
    over_18 = False
    try:
        subreddit = reddit.subreddit(subreddit_name)
        over_18 = subreddit.over18
    except:
        print(f"could not find subreddit '{subreddit_name}'")
        with open("invalid_subreddits.csv", 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([subreddit_name])
        continue
    
    if over_18:
        with open("over_18_subreddits.csv", 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([subreddit_name])
            continue
    links = find_links_in_submissions(subreddit, cutoff_date, save_to=edges_csv)
    
    for link in links:
        linked_subreddit = link["subreddit"]
        if linked_subreddit in visited_subreddits:
            continue
        subreddit_visit_queue.add(linked_subreddit)
    print(f"subreddit '{subreddit_name}' done")

Processing subreddit 'home' (from popular)
subreddit 'home' done
Processing subreddit 'mic'
could not find subreddit 'mic'
Processing subreddit 'askreddit' (from popular)
subreddit 'askreddit' done
Processing subreddit 'pics' (from popular)
subreddit 'pics' done
Processing subreddit 'boiler'
subreddit 'boiler' done
Processing subreddit 'foyer'
could not find subreddit 'foyer'
Processing subreddit '5013011561'
could not find subreddit '5013011561'
Processing subreddit 'dryer'
could not find subreddit 'dryer'
Processing subreddit 'camouflage'
subreddit 'camouflage' done
Processing subreddit 'od'
could not find subreddit 'od'
Processing subreddit 'concrete'
subreddit 'concrete' done
Processing subreddit 'baldursgate3' (from popular)
subreddit 'baldursgate3' done
Processing subreddit 'father'
could not find subreddit 'father'
Processing subreddit 'troweled'
could not find subreddit 'troweled'
Processing subreddit 'airsoftmarket'
subreddit 'airsoftmarket' done
Processing subreddit 'facepalm

subreddit 'keyboard' done
Processing subreddit 'typing'
subreddit 'typing' done
Processing subreddit 'ww2memes'
subreddit 'ww2memes' done
Processing subreddit 'foster'
could not find subreddit 'foster'
Processing subreddit 'sipstea' (from popular)
subreddit 'sipstea' done
Processing subreddit 'independent'
subreddit 'independent' done
Processing subreddit 'file'
subreddit 'file' done
Processing subreddit 'bigmemermaan1'
could not find subreddit 'bigmemermaan1'
Processing subreddit 'repacker'
could not find subreddit 'repacker'
Processing subreddit 'warzone'
subreddit 'warzone' done
Processing subreddit 'intermediate'
could not find subreddit 'intermediate'
Processing subreddit 'pia'
subreddit 'pia' done
Processing subreddit 'furniture'
subreddit 'furniture' done
Processing subreddit 'bestofredditorupdates' (from popular)
subreddit 'bestofredditorupdates' done
Processing subreddit 'white'
subreddit 'white' done
Processing subreddit 'pretty_analysis6860'
could not find subreddit 'pretty_

could not find subreddit 'boot_resolution_very_low'
Processing subreddit 'helck'
subreddit 'helck' done
Processing subreddit 'asksouthafrica'
subreddit 'asksouthafrica' done
Processing subreddit 'lyft'
subreddit 'lyft' done
Processing subreddit 'ghoststories'
subreddit 'ghoststories' done
Processing subreddit 'good4goblin'
could not find subreddit 'good4goblin'
Processing subreddit 'rpghorrorstories'
subreddit 'rpghorrorstories' done
Processing subreddit 'north'
subreddit 'north' done
Processing subreddit 'f1circlejerk'
subreddit 'f1circlejerk' done
Processing subreddit 'meirl' (from popular)
subreddit 'meirl' done
Processing subreddit 'nba' (from popular)
subreddit 'nba' done
Processing subreddit 'dsl'
could not find subreddit 'dsl'
Processing subreddit 'oddlysatisfying' (from popular)
subreddit 'oddlysatisfying' done
Processing subreddit 'results'
subreddit 'results' done
Processing subreddit 'datamodelling'
subreddit 'datamodelling' done
Processing subreddit 'technology' (from popul

subreddit 'corsair' done
Processing subreddit 'family'
subreddit 'family' done
Processing subreddit '8574'
could not find subreddit '8574'
Processing subreddit 'oops_mother_comes_out_of_the_closet_and_then'
could not find subreddit 'oops_mother_comes_out_of_the_closet_and_then'
Processing subreddit 'gold'
subreddit 'gold' done
Processing subreddit 'interestingasfuck' (from popular)
subreddit 'interestingasfuck' done
Processing subreddit 'show'
could not find subreddit 'show'
Processing subreddit 'legendary'
subreddit 'legendary' done
Processing subreddit 'announcing_changes_to_spoiler_tags'
could not find subreddit 'announcing_changes_to_spoiler_tags'
Processing subreddit 'feedthebeast' (from popular)
subreddit 'feedthebeast' done
Processing subreddit 'orlandomagic'
subreddit 'orlandomagic' done
Processing subreddit 'frugal'
subreddit 'frugal' done
Processing subreddit 'askuk'
subreddit 'askuk' done
Processing subreddit 'resto'
could not find subreddit 'resto'
Processing subreddit 'mon

subreddit 'dankmemes' done
Processing subreddit 'norway'
subreddit 'norway' done
Processing subreddit '2040'
could not find subreddit '2040'
Processing subreddit 'swiftiemerch'
subreddit 'swiftiemerch' done
Processing subreddit 'jujutsufolk' (from popular)
subreddit 'jujutsufolk' done
Processing subreddit 'trustedinstaller'
could not find subreddit 'trustedinstaller'
Processing subreddit 'holup' (from popular)
subreddit 'holup' done
Processing subreddit 'stablediffusion' (from popular)
subreddit 'stablediffusion' done
Processing subreddit 'me_irl' (from popular)
subreddit 'me_irl' done
Processing subreddit 'endeavouros'
subreddit 'endeavouros' done
Processing subreddit 'cardano'
subreddit 'cardano' done
Processing subreddit 'dynastycw'
subreddit 'dynastycw' done
Processing subreddit 'nextfuckinglevel' (from popular)
subreddit 'nextfuckinglevel' done
Processing subreddit 'weapon'
could not find subreddit 'weapon'
Processing subreddit 'saopaulo'
subreddit 'saopaulo' done
Processing subre

...

subreddit 'gunners' done
Processing subreddit '130691'
could not find subreddit '130691'
Processing subreddit 'elecciones'
could not find subreddit 'elecciones'
Processing subreddit 'pvp'
subreddit 'pvp' done
Processing subreddit 'kilo_g_looked_up'
could not find subreddit 'kilo_g_looked_up'
Processing subreddit 'unccharlotte'
subreddit 'unccharlotte' done
Processing subreddit 'interviewers'
could not find subreddit 'interviewers'
Processing subreddit 'namodeinorauses'
could not find subreddit 'namodeinorauses'
Processing subreddit 'notrequebec'
subreddit 'notrequebec' done
Processing subreddit 'spanish'
subreddit 'spanish' done
Processing subreddit 'tenderfoottuesday'
subreddit 'tenderfoottuesday' done
Processing subreddit 'capitolconsequences'
subreddit 'capitolconsequences' done
Processing subreddit 'discounts'
could not find subreddit 'discounts'
Processing subreddit 'productivity' (from popular)
subreddit 'productivity' done
Processing subreddit 'fl_studio' (from popular)
subreddi

Processing subreddit 'request'
could not find subreddit 'request'
Processing subreddit 'antinatalism'
subreddit 'antinatalism' done
Processing subreddit 'gobuffs'
subreddit 'gobuffs' done
Processing subreddit 'sci'
subreddit 'sci' done
Processing subreddit 'jungleporn'
subreddit 'jungleporn' done
Processing subreddit 'gothamfc'
subreddit 'gothamfc' done
Processing subreddit 'nl'
subreddit 'nl' done
Processing subreddit 'snowy'
subreddit 'snowy' done
Processing subreddit 'beardporn'
Processing subreddit 'keeping'
could not find subreddit 'keeping'
Processing subreddit 'mattcolville'
subreddit 'mattcolville' done
Processing subreddit 'inthemorning'
subreddit 'inthemorning' done
Processing subreddit 'fragrance' (from popular)
subreddit 'fragrance' done
Processing subreddit 'verticalbackgrounds'
subreddit 'verticalbackgrounds' done
Processing subreddit 'applewatch' (from popular)
subreddit 'applewatch' done
Processing subreddit 'rakna'
could not find subreddit 'rakna'
Processing subreddit 

subreddit '7daystodie' done
Processing subreddit 'tronscript'
subreddit 'tronscript' done
Processing subreddit 'empiredidnothingwrong'
subreddit 'empiredidnothingwrong' done
Processing subreddit 'dialogos'
subreddit 'dialogos' done
Processing subreddit 'justnofamily'
subreddit 'justnofamily' done
Processing subreddit 'libros'
subreddit 'libros' done
Processing subreddit 'outdoors'
subreddit 'outdoors' done
Processing subreddit 'different_sob_story'
subreddit 'different_sob_story' done
Processing subreddit '411330'
could not find subreddit '411330'
Processing subreddit 'commandlinetools'
could not find subreddit 'commandlinetools'
Processing subreddit 'norge' (from popular)
subreddit 'norge' done
Processing subreddit 'bg3' (from popular)
subreddit 'bg3' done
Processing subreddit 'characterrant' (from popular)
subreddit 'characterrant' done
Processing subreddit 'turtles'
subreddit 'turtles' done
Processing subreddit 'streams'
subreddit 'streams' done
Processing subreddit '690829'
could n

subreddit 'pulitzercomments' done
Processing subreddit 'moviesuggestions'
subreddit 'moviesuggestions' done
Processing subreddit 'resannouncements'
subreddit 'resannouncements' done
Processing subreddit '678554'
could not find subreddit '678554'
Processing subreddit 'youngpeopleyoutube' (from popular)
subreddit 'youngpeopleyoutube' done
Processing subreddit 'jnmil'
subreddit 'jnmil' done
Processing subreddit 'blood'
could not find subreddit 'blood'
Processing subreddit 'shadowban'
subreddit 'shadowban' done
Processing subreddit 'mauerstrassenwetten'
subreddit 'mauerstrassenwetten' done
Processing subreddit '14494'
could not find subreddit '14494'
Processing subreddit 'meekofurever'
subreddit 'meekofurever' done
Processing subreddit 'the_lex_interview_with_robert_kennedy_jr_was'
could not find subreddit 'the_lex_interview_with_robert_kennedy_jr_was'
Processing subreddit 'newenglandnsw'
subreddit 'newenglandnsw' done
Processing subreddit 'ti84hacks'
subreddit 'ti84hacks' done
Processing 

ServerError: received 500 HTTP response